In [ ]:
#We have a dataset containing images taken from boat for surveillance
#3778 noisy images containing six species of fishes and one category that specifies images with no fish
#and one category for other fishes
#This dataset is taken from www.kaggle.com provided by Nature conservancy, California

#Import all the packages needed

import numpy as np

import cv2

import os

from PIL import Image

import scipy

import matplotlib.pyplot as plt

import pandas as pd

from sklearn.cross_validation import train_test_split

from sklearn.metrics import confusion_matrix

from sklearn import metrics

from sklearn.utils import shuffle

from sklearn import preprocessing

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import Dropout

from keras.layers import Flatten

from keras.constraints import maxnorm

from keras.optimizers import SGD

from keras.layers.convolutional import Convolution2D

from keras.layers.convolutional import MaxPooling2D

from keras.utils import np_utils

from keras import backend as K


#Made a matrix by flattening each image in which each row is a image from the dataset

path = 'Desktop/Problem/Images'

p = ['ALB','BET','DOL','LAG','NoF','OTHER','SHARK','YFT']
L=[]

#Made a matrix for each category in the dataset

for i in range(8):
    listing = os.listdir(path+'/'+p[i])
    p[i] = np.array([np.array(cv2.imread(path+'/'+p[i]+'/'+file,0)).flatten()for file in listing])
    L.append(len(listing))

#Concatenated each matrix into one matrix
    
    
M = np.concatenate((p[0],p[1]))
for i in range(2,6):
    M = np.concatenate((M,p[i]))
label = np.ones(3513,dtype = int)

#Labelled each row with values 1 to 8

for i in range(1,8):
	label[0:1074]=0
	label[sum(L[0:i])+1:sum(L[0:i+1])+1] = i
print (label)


#Shuffled the data for better perfomance

data,Label = shuffle(M,label,random_state=2)

train_data = [data,Label]

X,Y = [train_data[0],train_data[1]]


#Splitted the data into Training set, Test set and Validation set

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.4,random_state=4)

x_test,x_validation,y_test,y_validation=train_test_split(X_test,Y_test,test_size=0.5,random_state=4)

print (X_train.shape)

X_train = X_train.reshape((X_train.shape[0],256,256,1))

x_validation = x_validation.reshape((x_validation.shape[0],256,256,1))

x_test =x_test.reshape((x_test.shape[0],256,256,1))

X_train = X_train.astype('float32')

x_validation = x_validation.astype('float32')

x_test = x_test.astype('float32')

#Normalised each of the pixel values in the training, test and validation sets

X_train=X_train/255

x_test = x_test / 255

x_validation = x_validation/255

#Vectorized each label set into a vector of dimension 8

Y_train = np_utils.to_categorical(Y_train,8)

y_validation = np_utils.to_categorical(y_validation,8)

w_test = np_utils.to_categorical(y_test,8)

#Constructed the model of Convolutional Neural Network

def fish_model():
    model = Sequential()
    model.add(Convolution2D(40, 3, 3,  subsample=(2,2), input_shape=(256, 256, 1), activation='relu', border_mode='same'))
    model.add(Convolution2D(40, 3, 3, activation='relu', border_mode='same'))
    model.add(Convolution2D(40, 3, 3, activation='relu', border_mode='same'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(1,1)))
    model.add(Flatten())
    model.add(Dense(6, activation='softmax'))
#Kept Stochastic gradient descent algorithm for wait updation
    epochs = 5
    lrate = 1
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.5, decay=decay, nesterov=False)
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['categorical_accuracy'])
    return model

model= fish_model()
print(model.summary())


#Fit the model to the training set and validation in each epoch

history = model.fit(X_train, Y_train, validation_data=(x_validation, y_validation), nb_epoch=16, batch_size=4)
print (history.history)
scores = model.evaluate(x_test,w_test,verbose=0)
print('Test Accuracy:',scores[1]*100)

#Predicted for the test dataset 

prediction=model.predict_proba(x_test, batch_size=4,verbose=1)


#Made Confusion matrix, Precison-recall tables
p = []
for i in range(703):
    m=max(prediction[i])
    a=list(prediction[i]).index(m)
    p.append(a)
print(confusion_matrix(y_test,p))
print(metrics.classification_report(y_test,p, target_names = ['0','1','2','3','4','5']))





In [3]:
from textblob import TextBlob
from collections import defaultdict
#import seaborn as sb

# defaultdict(int) is exactly like a Python dictionary, but you can specify the default value of every key;
# If 'int', then default value is 0; if 'str', default value is ''; if list, default value is []
# We need 'int', because we're counting the number for each tag, and starting value is 0.

d = defaultdict(int)

with open('spam.csv','r') as f:
	# Taking each line in the file
	for line in f:
		text = TextBlob(line)

		tagged_list = text.tags
		# Now tagged_list is a list like [('Python', 'NNP'), ('is', 'VBZ'), ('a', 'DT'), ('high-level', 'JJ'), ('general-purpose', 'JJ'), ('programming', 'NN'), ('language', 'NN')]

		for tuplepair in tagged_list:
			tag = tuplepair[1]
			d[tag] += 1

d = dict(d)
#sb.barplot(x=list(d.keys()),y=list(d.values()))

# Now, d contains the counts for all POS tags. Now make a barplot using those counts.

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 606-607: invalid continuation byte